In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML  # For Jupyter notebooks
import random
import sys
from itertools import pairwise
import requests
from io import StringIO
import time

ArgumentError: ArgumentError: Package networkx not found in current path.
- Run `import Pkg; Pkg.add("networkx")` to install the networkx package.

In [2]:

class PathAnimator:
    def __init__(self, G, path_edges, pos=None, seed=42, figsize=(8, 6)):
        self.G = G
        self.path_edges = path_edges
        self.path_nodes = self._get_nodes_from_edges(path_edges)
        self.pos = pos if pos is not None else nx.spring_layout(G, seed=seed)
        self.fig, self.ax = plt.subplots(figsize=figsize)
        self.ani = None

    def _get_nodes_from_edges(self, edges):
        if not edges:
            return []
        nodes = [edges[0][0]]
        current_node = edges[0][0]
        for u, v in edges:
            if current_node == u:
                next_node = v
            elif current_node == v:
                next_node = u
            else:
                continue  # Skip edges not connected to the current path
            nodes.append(next_node)
            current_node = next_node
        return nodes

    def init_func(self):
        self.ax.clear()
        nx.draw_networkx_nodes(self.G, self.pos, ax=self.ax, node_color='lightblue', node_size=500)
        nx.draw_networkx_edges(self.G, self.pos, ax=self.ax, edge_color='gray', alpha=0.5)
        nx.draw_networkx_labels(self.G, self.pos, ax=self.ax)
        self.ax.set_title("Path Animation")
        return (self.ax,)

    def update_func(self, frame):
        self.ax.clear()
        nx.draw_networkx_nodes(self.G, self.pos, ax=self.ax, node_color='lightblue', node_size=500)
        nx.draw_networkx_edges(self.G, self.pos, ax=self.ax, edge_color='gray', alpha=0.3)
        nx.draw_networkx_labels(self.G, self.pos, ax=self.ax)

        if frame > 0:
            current_edges = self.path_edges[:frame]
            current_nodes = self.path_nodes[:frame+1]

            nx.draw_networkx_edges(self.G, self.pos, edgelist=current_edges, ax=self.ax,
                                   edge_color='red', width=2)
            nx.draw_networkx_nodes(self.G, self.pos, nodelist=current_nodes, ax=self.ax,
                                  node_color='red', node_size=500)

        self.ax.set_title(f"Step {frame}/{len(self.path_edges)}")
        return (self.ax,)

    def animate(self, interval=500):
        self.ani = FuncAnimation(
            self.fig,
            self.update_func,
            frames=len(self.path_edges) + 1,
            init_func=self.init_func,
            interval=interval,
            blit=False
        )
        plt.close(self.fig)
        return HTML(self.ani.to_jshtml())

UndefVarError: UndefVarError: `class` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [3]:
class DP_HamiltonianPath:
    def __init__(self, G, source, sink):
        self.G = G
        self.nodes = list(G.nodes())
        self.n = len(self.nodes)
        self.source = source
        self.sink = sink
        # Create node to index mapping for consistent indexing
        self.node_to_idx = {node: i for i, node in enumerate(self.nodes)}
        self.idx_to_node = {i: node for i, node in enumerate(self.nodes)}
        # DP table: dp[node_idx][mask] = True if we can reach sink from node_idx having visited nodes in mask
        self.dp = {}
        self.computed = set()

    def DP_compute(self, current_node, visited_mask):
        """
        Returns True if there exists a Hamiltonian path from current_node to sink
        having already visited the nodes represented by visited_mask

        Args:
            current_node: Current node index
            visited_mask: Bitmask representing already visited nodes
        """
        state = (current_node, visited_mask)

        # Return memoized result if already computed
        if state in self.computed:
            return self.dp.get(state, False)

        # Mark current node as visited
        new_visited_mask = visited_mask | (1 << current_node)

        # Base case: If we're at sink and have visited all nodes
        if current_node == self.node_to_idx[self.sink]:
            result = (new_visited_mask == (1 << self.n) - 1)
            self.dp[state] = result
            self.computed.add(state)
            return result

        # If we've visited all nodes but not at sink, impossible
        if new_visited_mask == (1 << self.n) - 1:
            self.dp[state] = False
            self.computed.add(state)
            return False

        # Try all unvisited neighbors
        result = False
        current_actual_node = self.idx_to_node[current_node]

        for neighbor in self.G.neighbors(current_actual_node):
            neighbor_idx = self.node_to_idx[neighbor]

            # If neighbor hasn't been visited yet
            if not (new_visited_mask & (1 << neighbor_idx)):
                if self.DP_compute(neighbor_idx, new_visited_mask):
                    result = True
                    break

        self.dp[state] = result
        self.computed.add(state)
        return result

    def exists(self):
        """Check if Hamiltonian path from source to sink exists"""
        source_idx = self.node_to_idx[self.source]
        # Start with empty visited set (source will be marked as visited in DP_compute)
        return self.DP_compute(source_idx, 0)

    def get_path(self):
        """Reconstruct the actual Hamiltonian path if it exists"""
        if not self.exists():
            return []

        path = []
        current_node_idx = self.node_to_idx[self.source]
        visited_mask = 0

        while True:
            current_actual_node = self.idx_to_node[current_node_idx]
            path.append(current_actual_node)

            # Mark current node as visited
            visited_mask |= (1 << current_node_idx)

            # If we reached the sink, we're done
            if current_actual_node == self.sink:
                break

            # Find next node in the path
            found_next = False
            for neighbor in self.G.neighbors(current_actual_node):
                neighbor_idx = self.node_to_idx[neighbor]

                # If neighbor hasn't been visited
                if not (visited_mask & (1 << neighbor_idx)):
                    # Check if going to this neighbor leads to a solution
                    if self.DP_compute(neighbor_idx, visited_mask):
                        current_node_idx = neighbor_idx
                        found_next = True
                        break

            if not found_next:
                return []  # This shouldn't happen if exists() returned True

        return path


# Analysis and Testing Code
def create_test_graph():
    """Create a simple test graph for demonstration"""
    import networkx as nx

    # Create a simple path graph: 0 - 1 - 2 - 3
    G = nx.Graph()
    G.add_edges_from([(0, 1), (1, 2), (2, 3)])
    return G

def test_hamiltonian_path():
    """Test the implementation with various cases"""
    import networkx as nx

    print("=== Testing Hamiltonian Path DP Implementation ===\n")

    # Test 1: Simple path graph
    print("Test 1: Path graph 0-1-2-3")
    G1 = nx.Graph()
    G1.add_edges_from([(0, 1), (1, 2), (2, 3)])

    hp1 = DP_HamiltonianPath(G1, 0, 3)
    exists1 = hp1.exists()
    path1 = hp1.get_path()
    print(f"Hamiltonian path exists: {exists1}")
    print(f"Path: {path1}")
    print()

    # Test 2: Complete graph K4
    print("Test 2: Complete graph K4")
    G2 = nx.complete_graph(4)

    hp2 = DP_HamiltonianPath(G2, 0, 3)
    exists2 = hp2.exists()
    path2 = hp2.get_path()
    print(f"Hamiltonian path exists: {exists2}")
    print(f"Path: {path2}")
    print()

    # Test 3: Disconnected graph
    print("Test 3: Disconnected graph")
    G3 = nx.Graph()
    G3.add_edges_from([(0, 1), (2, 3)])  # Two separate edges

    hp3 = DP_HamiltonianPath(G3, 0, 3)
    exists3 = hp3.exists()
    path3 = hp3.get_path()
    print(f"Hamiltonian path exists: {exists3}")
    print(f"Path: {path3}")
    print()

if __name__ == "__main__":
    test_hamiltonian_path()

UndefVarError: UndefVarError: `class` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [4]:
G = nx.Graph()

# Add nodes (example: 16 nodes)
G.add_nodes_from(range(16))

# Add edges (example: manually or randomly)
# Here's an example where we manually add a Hamiltonian path and add a few more edges
hamiltonian_path_nodes = [0, 1, 2, 3, 7, 6, 5, 4, 8, 9, 10, 11, 15, 14, 13, 12]
hamiltonian_path_edges = list(pairwise(hamiltonian_path_nodes))
G.add_edges_from(hamiltonian_path_edges)

# Add a few extra edges (optional)
G.add_edges_from([
    (0, 5), (2, 6), (8, 12), (10, 14), (1, 9),(3 ,5),(0,8),(2,4)
])
g=DP_HamiltonianPath(G,0,12)
path=g.get_path()
print(path)
path_edges = list(pairwise(path))

UndefVarError: UndefVarError: `nx` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
p=PathAnimator(G,path_edges)
p.animate()

UndefVarError: UndefVarError: `PathAnimator` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
class LCT:
  def __init__(self, n):
    # Initialize the Link-Cut Tree with n nodes
    self.n = n
    self.ch = [[0, 0] for i in range(n + 1)]  # Children array
    self.fa = [0 for i in range(n + 1)]  # Parent array
    self.rev = [0 for i in range(n + 1)]  # Reverse flag array

  def isr(self, a):
    # Check if node a is a root of its splay tree
    return not (self.ch[self.fa[a]][0] == a or self.ch[self.fa[a]][1] == a)

  def pushdown(self, a):
    # Propagate the reverse operation down the tree
    if self.rev[a]:
      self.rev[self.ch[a][0]] ^= 1
      self.rev[self.ch[a][1]] ^= 1
      self.ch[a][0], self.ch[a][1] = self.ch[a][1], self.ch[a][0]  # Swap children
      self.rev[a] = 0

  def push(self, a):
    # Push changes down to ensure the node a is up-to-date
    if not self.isr(a):
      self.push(self.fa[a])
    self.pushdown(a)

  def rotate(self, a):
    # Perform a rotation on node a
    f = self.fa[a]
    gf = self.fa[f]
    tp = self.ch[f][1] == a
    son = self.ch[a][tp ^ 1]
    if not self.isr(f):
      self.ch[gf][self.ch[gf][1] == f] = a
    self.fa[a] = gf
    self.ch[f][tp] = son
    if son:
      self.fa[son] = f
    self.ch[a][tp ^ 1] = f
    self.fa[f] = a

  def splay(self, a):
    self.push(a)
    while not self.isr(a):
        f = self.fa[a]
        gf = self.fa[f]
        if self.isr(f):
            self.rotate(a)
        else:
            tp1 = (self.ch[gf][1] == f)
            tp2 = (self.ch[f][1] == a)
            if tp1 == tp2:
                self.rotate(f)
                self.rotate(a)
            else:
                self.rotate(a)
                self.rotate(a)


  def access(self, a):
    # Make node a the root of the preferred path
    pr = a
    self.splay(a)
    self.ch[a][1] = 0
    while True:
      if not(self.fa[a]):
        break
      u = self.fa[a]
      self.splay(u)
      self.ch[u][1]=a
      a=u
    self.splay(pr)

  def makeroot(self, a):
    # Make node a the root of the entire tree
    self.access(a)
    self.rev[a] ^= 1

  def link(self, a, b):
    # Link node a to node b
    self.makeroot(a)
    self.fa[a] = b

  def cut(self, a, b):
    # Cut the connection between node a and node b
    self.makeroot(a)
    self.access(b)
    self.fa[a] = 0
    self.ch[b][0]=0

  def fdr(self, a):
    # Find the root of the tree containing node a
    self.access(a)
    while True:
      self.pushdown(a)
      if self.ch[a][0]:
        a = self.ch[a][0]
      else:
        self.splay(a)
        return a


UndefVarError: UndefVarError: `class` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
def hamiltonian(n, edges, mx_ch=-1):
    # mx_ch : max number of adding/replacing; default is (n + 100) * (n + 50)
    # n : number of vertices (1-indexed)
    # edges : list of (u, v) pairs (1-indexed)
    # Returns a list of vertices forming a Hamiltonian path, empty list if not found
    for i in range(len(edges)):
        u,v=edges[i]
        edges[i]=(u+1,v+1)
    out = [0] * (n + 1)
    ine = [0] * (n + 1)
    lc = LCT(n)
    if mx_ch == -1:
        mx_ch = (n + 100) * (n + 50)
    frm = [[] for _ in range(n + 1)]
    to = [[] for _ in range(n + 1)]

    # Process edges assuming they are 1-indexed
    for u, v in edges:
        frm[u].append(v)
        to[v].append(u)

    canin = set(range(1, n+1))
    canout = set(range(1, n+1))
    tot = 0

    while mx_ch >= 0:
        eg = []
        for v in canout:
            for s in frm[v]:
                if ine[s] == 0:
                    continue
                eg.append((v, s))
        for v in canin:
            for s in to[v]:
                eg.append((s, v))
        random.shuffle(eg)
        if not eg:
            break
        for u, v in eg:
            mx_ch -= 1
            if ine[v] and out[u]:
                continue
            if lc.fdr(u) == lc.fdr(v):
                continue
            if ine[v] or out[u]:
                if random.choice([0, 1]):
                    continue
            if ine[v] == 0 and out[u] == 0:
                tot += 1
            if ine[v]:
                lc.cut(ine[v], v)
                canin.add(v)
                canout.add(ine[v])
                out[ine[v]] = 0
                ine[v] = 0
            if out[u]:
                lc.cut(u, out[u])
                canin.add(out[u])
                canout.add(u)
                ine[out[u]] = 0
                out[u] = 0
            lc.link(u, v)
            canin.discard(v)
            canout.discard(u)
            ine[v] = u
            out[u] = v

    if tot == n - 1:
        cur = []
        # Find the starting node (ine[i] == 0)
        start = None
        for i in range(1, n+1):
            if ine[i] == 0:
                start = i
                break
        if start is not None:
            pl = start
            while pl != 0:
                cur.append(pl)
                pl = out[pl]
        if len(cur)==n:
            for i in range(n):
                cur[i]-=1
            return cur
    return []

UndefVarError: UndefVarError: `def` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [8]:
G = nx.Graph()

# Add nodes (example: 16 nodes)
G.add_nodes_from(range(10))

    # Add edges (example: manually or randomly)
    # Here's an example where we manually add a Hamiltonian path and add a few more edges
l1=[(i,j) for  i in range(10)  for j in range(i+1,10)]
G.add_edges_from(l1)
g=list(pairwise(hamiltonian(10,l1)))
print(l1)
print(g)

UndefVarError: UndefVarError: `nx` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [9]:

p=PathAnimator(G,g)
p.animate()

UndefVarError: UndefVarError: `PathAnimator` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [10]:


# Direct download link (modified from your Google Drive URL)
file_url = "https://drive.google.com/uc?id=1uvxg6a3RH7rm00xDFrpT99Yin5Bakj36&export=download"

# Download the file
response = requests.get(file_url)
response.raise_for_status()  # Check for errors

# Read the file content line by line
with StringIO(response.text) as file:
    n, m = map(int, file.readline().split())  # Read vertices (n) and edges (m)
    edges = [tuple(map(int, file.readline().split())) for _ in range(m)]  # Read edges

print("Number of vertices (n):", n)
print("Number of edges (m):", m)
print("Edges:", edges)

# Now you can use `edges` in your graph code
# Example:
G.add_edges_from(edges)
g = list(pairwise(hamiltonian(n, edges)))
p = PathAnimator(G, g)
p.animate()

UndefVarError: UndefVarError: `requests` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [11]:


# Direct download link (modified from your Google Drive URL)
file_url = "https://drive.google.com/uc?id=10b_it9tK7d3nJn08zKov0R_eoo0k7sQU&export=download"

# Download the file
response = requests.get(file_url)
response.raise_for_status()  # Check for errors

# Read the file content line by line
with StringIO(response.text) as file:
    n, m = map(int, file.readline().split())  # Read vertices (n) and edges (m)
    edges = [tuple(map(int, file.readline().split())) for _ in range(m)]  # Read edges

print("Number of vertices (n):", n)
print("Number of edges (m):", m)
print("Edges:", edges)

# Now you can use `edges` in your graph code
# Example:
G.add_edges_from(edges)
g = list(pairwise(hamiltonian(n, edges)))
p = PathAnimator(G, g)
p.animate()

UndefVarError: UndefVarError: `requests` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [12]:
edges = [
    (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11),
    (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17),
    (17, 18), (18, 19), (19, 0), (0, 1), (1, 2), (2, 3)
]

# Convert to set for quick lookup
edge_set = set(edges + [(b, a) for (a, b) in edges])
num_nodes = 20
extra_edges = 5
added = 0

while added < extra_edges:
    u = random.randint(0, num_nodes - 1)
    v = random.randint(0, num_nodes - 1)
    if u != v and (u, v) not in edge_set and (v, u) not in edge_set:
        edges.append((u, v))
        edge_set.add((u, v))
        edge_set.add((v, u))
        added += 1

# Final edge list with random edges added
print(edges)

Gg=nx.Graph()
G.add_nodes_from(range(20))
Gg.add_edges_from(edges)
g=list(pairwise(hamiltonian(20,edges)))
p=PathAnimator(Gg,g)
p.animate()

UndefVarError: UndefVarError: `set` not defined in `Main`
Suggestion: check for spelling errors or missing imports.